In [ ]:
# from  tensorflow.contrib.learn import DNNClassifier, SKCompat
from scribe_classifier.data.canada import AllCodes, TitleSet, SimpleModel, CombinedModels
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [64]:
all_codes = AllCodes.load_from_pickle("./source_data/pickles/canada/tidy_sets/all_codes.P", is_path=True)

target_level = 3
emptyset_label = "NA"

code_vec = sorted(all_codes.get_codes_for_level(target_level=target_level))
if emptyset_label is not None:
    code_vec.append(emptyset_label)

print(code_vec)
lblenc = LabelEncoder()
lblenc.fit(code_vec)

mdl_sgd = SimpleModel.load_from_pickle(
    "./source_data/pickles/canada/trained_models/simple.lvl%d.P" % target_level,
    is_path=True
)
mdl_nb = SimpleModel.load_from_pickle(
    "./source_data/pickles/canada/trained_models/simple.lvl%d.bayes.P" % target_level,
    is_path=True
)
print(lblenc.transform(code_vec))

['001', '011', '012', '013', '021', '031', '041', '042', '043', '051', '060', '062', '063', '065', '071', '073', '081', '082', '091', '111', '112', '121', '122', '124', '125', '131', '141', '142', '143', '145', '151', '152', '211', '212', '213', '214', '215', '216', '217', '221', '222', '223', '224', '225', '226', '227', '228', '301', '311', '312', '313', '314', '321', '322', '323', '341', '401', '402', '403', '411', '415', '416', '421', '431', '441', '442', '511', '512', '513', '521', '522', '523', '524', '525', '621', '622', '623', '631', '632', '633', '634', '641', '642', '651', '652', '653', '654', '655', '656', '661', '662', '671', '672', '673', '674', '720', '723', '724', '725', '727', '728', '729', '730', '731', '732', '733', '736', '737', '738', '744', '745', '751', '752', '753', '761', '762', '821', '822', '823', '824', '825', '826', '841', '842', '843', '844', '861', '921', '922', '923', '924', '941', '942', '943', '944', '946', '947', '952', '953', '961', 'NA']
[  0   1   2 

In [3]:
test = TitleSet.load_from_pickle('source_data/pickles/canada/test_sets/test.set.lvl%d.P' % target_level, is_path=True)
valid = TitleSet.load_from_pickle('source_data/pickles/canada/test_sets/valid.set.lvl%d.P' % target_level, is_path=True)
train = TitleSet.load_from_pickle('source_data/pickles/canada/test_sets/train.set.lvl%d.P' % target_level, is_path=True)

valid = valid.copy_and_append_empty_string_class(label=emptyset_label, prop_records=0.25)
test = test.copy_and_append_empty_string_class(label=emptyset_label, prop_records=0.25)

In [4]:
cmb_mdl = CombinedModels(
    target_level=target_level, 
    emptyset_label="NA",
    trained_simple_sgd=mdl_sgd, 
    trained_simple_multinom_nb=mdl_nb, 
    all_codes=all_codes)
# cmb_mdl.fit_titleset(train)

In [65]:
train_title_vec = train.get_title_vec()
train_code_vec = train.get_code_vec(target_level=target_level)

In [68]:
sgd_bag, nb_bag, sgd_predict, nb_proba = cmb_mdl.assemble_features(train_title_vec)
sgd_predict = sgd_predict.reshape(len(sgd_predict), 1)
print(sgd_bag.shape)
print(nb_bag.shape)
print(sgd_predict.shape)
print(nb_proba.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(21925, 6448)
(21925, 6448)
(21925, 1)
(21925, 141)


In [69]:
from tensorflow.contrib.layers import real_valued_column, sparse_column_with_integerized_feature
from tensorflow.contrib.layers import sparse_column_with_hash_bucket
from tensorflow.contrib.learn import SKCompat, DNNClassifier
import tensorflow as tf
import pandas as pd
import numpy as np

In [82]:
data_arr = np.concatenate((sgd_bag, nb_bag, sgd_predict, nb_proba), axis=1).astype(np.float32)
df = pd.DataFrame(data_arr)
df.rename(columns=str, inplace=True)

In [83]:
features = []
for i in range(data_arr.shape[1]):
    features.append(real_valued_column(column_name="%d" % i))

estimator = SKCompat(DNNClassifier(
    n_classes=len(train_code_vec),
    feature_columns=features,
    hidden_units=[1024, 512, 256],
    label_keys=train_code_vec))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe637c5abe0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpn_o2h646'}


In [84]:
print(lblenc.transform(train_code_vec))
estimator.fit(df, train_code_vec)

[134 134 134 ...,  29  81  35]


KeyError: '0'